In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import google.generativeai as genai
import torch
from transformers import AutoTokenizer, pipeline
import random
import time
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'google.generativeai'

In [ ]:
load_dotenv()

In [ ]:
api_key = os.getenv("GOOGLE_API_KEY")

In [ ]:
if not api_key:
    raise ValueError("Δεν βρέθηκε το API Key! Βεβαιώσου ότι έχεις φτιάξει το αρχείο .env")

genai.configure(api_key=api_key)

In [ ]:
gemini_model = genai.GenerativeModel('gemini-1.5-flash')
print("✅ Το Gemini ρυθμίστηκε επιτυχώς με το κρυφό κλειδί.")

In [ ]:
# Έλεγχος αν βλέπει την GPU
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
else:
    print("ΠΡΟΣΟΧΗ: Δεν βρέθηκε GPU. Ελέγξτε τους drivers.")

In [ ]:
# ΦΟΡΤΩΣΗ LLAMA-KRIKRI
model_id = "ilsp/Llama-Krikri-8b-instruct"

In [ ]:
print("Φόρτωση μοντέλου στην GPU...")

In [ ]:
try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    krikri_pipeline = pipeline(
        "text-generation",
        model=model_id,
        model_kwargs={
            "torch_dtype": torch.float16,
            "load_in_4bit": True,
            "low_cpu_mem_usage": True
        },
        device_map="auto",
        tokenizer=tokenizer,
    )
    print("Το Llama-KriKri φορτώθηκε επιτυχώς!")
except Exception as e:
    print(f"Σφάλμα φόρτωσης: {e}")

In [ ]:
INPUT_FILE = "greek_dataset.xlsx - Φύλλο1.csv"

# Διάβασμα αρχείου
try:
    # Δοκιμή ως CSV (σύμφωνα με το όνομα του αρχείου που ανέβασες)
    df_real = pd.read_csv(INPUT_FILE, on_bad_lines='skip')
except:
    # Fallback σε Excel engine αν χρειαστεί
    df_real = pd.read_excel(INPUT_FILE)


In [ ]:
# Επιλογή στήλης 'review' και καθαρισμός
if 'review' in df_real.columns:
    df_real = df_real[['review']].dropna().copy()
else:
    # Αν για κάποιο λόγο δεν βρει το όνομα, παίρνουμε τη 2η στήλη
    print("Η στήλη 'review' δεν βρέθηκε, χρησιμοποιείται η 2η στήλη.")
    df_real = df_real.iloc[:, 1].to_frame(name='review').dropna()

In [ ]:
# Ανάθεση Label
df_real['label'] = 'real'

print(f"Σύνολο Real Reviews: {len(df_real)}")
df_real.head(3)

In [ ]:
def generate_fake_gemini():
    """Παραγωγή από Gemini"""
    try:
        sentiment = random.choice(["θετική", "αρνητική"])
        # Prompt σχεδιασμένο για fake reviews
        prompt = (
            f"Γράψε μια σύντομη {sentiment} κριτική (review) στα Ελληνικά για ένα προϊόν τεχνολογίας (π.χ. κινητό, laptop, gadget). "
            "Γράψε την σαν να είσαι ένας απλός χρήστης που αγόρασε από e-shop. "
            "Μην βάλεις τίτλο, μόνο το κυρίως κείμενο. Μην βάλεις εισαγωγικά."
        )
        response = gemini_model.generate_content(prompt)
        return response.text.strip().replace('"', '') # Καθαρισμός από τυχόν εισαγωγικά
    except:
        time.sleep(1)
        return None

In [ ]:
def generate_fake_krikri():
    """Παραγωγή από Llama-KriKri"""
    try:
        sentiment = random.choice(["θετική", "αρνητική"])

        # Chat format που καταλαβαίνει το μοντέλο instruct
        messages = [
            {"role": "user", "content": f"Γράψε μια σύντομη {sentiment} κριτική προϊόντος στα ελληνικά για ένα ηλεκτρονικό κατάστημα."}
        ]

        # Προετοιμασία prompt μέσω του tokenizer
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        outputs = krikri_pipeline(
            prompt,
            max_new_tokens=120,    # Μήκος κειμένου
            do_sample=True,        # Δημιουργικότητα
            temperature=0.8,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

        # Καθαρισμός του αποτελέσματος (αφαιρούμε το prompt)
        generated_text = outputs[0]["generated_text"]

        # Συνήθως το instruct μοντέλο βάζει ειδικά tokens, προσπαθούμε να πάρουμε το τελευταίο κομμάτι
        if "<|assistant|>" in generated_text:
            return generated_text.split("<|assistant|>")[-1].strip()
        else:
            # Fallback καθαρισμός
            return generated_text.replace(prompt, "").strip()

    except Exception as e:
        print(f"KriKri Error: {e}")
        return None

In [ ]:
num_real = len(df_real)
num_fake_needed = num_real

# Μοιράζουμε στη μέση
half = num_fake_needed // 2
fake_reviews = []

print(f"Στόχος: {num_fake_needed} fake reviews (Gemini: {half}, KriKri: {num_fake_needed - half})")

In [ ]:
# --- 1. Gemini Loop ---
print("Generating with Gemini...")
for _ in tqdm(range(half)):
    res = generate_fake_gemini()
    if res:
        fake_reviews.append(res)
    time.sleep(0.5) # Rate limiting protection

In [ ]:
# --- 2. KriKri Loop ---
print("Generating with Llama-KriKri...")
remaining = num_fake_needed - len(fake_reviews)

for _ in tqdm(range(remaining)):
    res = generate_fake_krikri()
    if res:
        fake_reviews.append(res)

In [ ]:
# Δημιουργία DataFrame
df_fake = pd.DataFrame(fake_reviews, columns=['review'])
df_fake['label'] = 'fake'

print(f"Ολοκληρώθηκε! Παρήχθησαν {len(df_fake)} fake reviews.")

In [ ]:
# Ένωση
df_final = pd.concat([df_real, df_fake], ignore_index=True)

In [ ]:
# Shuffle (Ανακάτεμα)
df_final = df_final.sample(frac=1).reset_index(drop=True)

In [ ]:
# Αποθήκευση
OUTPUT_FILE = "greek_fake_reviews_dataset.xlsx"
df_final.to_excel(OUTPUT_FILE, index=False)

In [ ]:
print(f"Το αρχείο σώθηκε ως: {OUTPUT_FILE}")
print("Κατανομή:")
print(df_final['label'].value_counts())